In [420]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory



In [421]:
clients_df = pd.read_csv("Proyecto_A_Caso2/clients.csv")
depots_df = pd.read_csv("Proyecto_A_Caso2/depots.csv")
vehicles_df = pd.read_csv("Proyecto_A_Caso2/vehicles.csv")



In [422]:
print(clients_df.head())
print(depots_df.head())
print(vehicles_df.head())


   ClientID  LocationID  Product  Longitude  Latitude
0         1          13       12 -74.196992  4.632553
1         2          14       15 -74.155037  4.601328
2         3          15       15 -74.101787  4.732421
3         4          16        6 -74.194862  4.638612
4         5          17        5 -74.110272  4.727692
   DepotID  LocationID  Longitude  Latitude  Capacity
0        1           1 -74.081242  4.750212         8
1        2           2 -74.109934  4.536383        10
2        3           3 -74.038548  4.792926         0
3        4           4 -74.067069  4.721678         4
4        5           5 -74.138263  4.607707        28
  VehicleType    Capacity        Range
0     Gas Car  131.921140   145.852071
1          EV  108.435620  1304.605971
2          EV   91.504255   953.172609
3       drone   32.896064    17.302304
4       drone   22.652628    16.627680


In [423]:
C=list(depots_df["DepotID"])
CAP=dict(zip(depots_df["DepotID"],depots_df["Capacity"]))


In [424]:
print("Conjunto C:", C)
print("Capacidades CAP_i:", CAP)


Conjunto C: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Capacidades CAP_i: {1: 8, 2: 10, 3: 0, 4: 4, 5: 28, 6: 3, 7: 0, 8: 10, 9: 43, 10: 1, 11: 16, 12: 18}


In [425]:
K=list(clients_df["ClientID"])
DEM=dict(zip(clients_df["ClientID"],clients_df["Product"]))  # Esta es la demanda que solicita ese cliente

In [426]:
print("Conjunto K: clientes", K)
print("Capacidades Demanda:", DEM)

Conjunto K: clientes [1, 2, 3, 4, 5, 6, 7, 8, 9]
Capacidades Demanda: {1: 12, 2: 15, 3: 15, 4: 6, 5: 5, 6: 11, 7: 12, 8: 10, 9: 15}


Conjunto para la distancia

In [427]:
def dist_haversiana(lon1,lat1,lon2,lat2):
  R=6371.0 #radio tierra
  lon1,lat1,lon2,lat2=map(np.radians,[lon1,lat1,lon2,lat2])
  dlon=lon2-lon1
  dlat=lat2-lat1
  a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
  c = 2 * np.arcsin(np.sqrt(a))
  return R * c



In [428]:
DIST={}
for i,depot in depots_df.iterrows():
  for j,client in clients_df.iterrows():
    di=depot["DepotID"]
    ci=client["ClientID"]
    distance=dist_haversiana(depot["Longitude"],depot["Latitude"],
                             client["Longitude"],client["Latitude"]
                             )
    DIST[(di,ci)]=round(distance,2)


In [429]:
for key, val in list(DIST.items())[:5]:
  print("La distancia entre el centro de distribucion", key[0], "y el cliente", key[1], "es de", val)

La distancia entre el centro de distribucion 1.0 y el cliente 1.0 es de 18.32
La distancia entre el centro de distribucion 1.0 y el cliente 2.0 es de 18.46
La distancia entre el centro de distribucion 1.0 y el cliente 3.0 es de 3.02
La distancia entre el centro de distribucion 1.0 y el cliente 4.0 es de 17.68
La distancia entre el centro de distribucion 1.0 y el cliente 5.0 es de 4.08


Procesamiento de vehiculos

In [430]:
vehicles_df

,VehicleType,Capacity,Range
0,Gas Car,131.921140,145.852071
1,EV,108.435620,1304.605971
2,EV,91.504255,953.172609
3,drone,32.896064,17.302304
4,drone,22.652628,16.627680
5,drone,22.682912,13.602811


In [431]:
# V=list(vehicles_df["VehicleType"])
V=list(vehicles_df.index)
Q = dict(zip(V, vehicles_df["Capacity"]))


R = dict(zip(V, vehicles_df["Range"]))



In [432]:
print("Vehículos:", V)
print("Capacidades:", Q)
print("Rangos:", R)


Vehículos: [0, 1, 2, 3, 4, 5]
Capacidades: {0: 131.9211396722696, 1: 108.4356199315333, 2: 91.50425520531196, 3: 32.896064077536955, 4: 22.65262807032524, 5: 22.682911535937688}
Rangos: {0: 145.85207096486445, 1: 1304.605971281605, 2: 953.172608610164, 3: 17.302304187458727, 4: 16.627680130757895, 5: 13.602810739289229}


In [433]:
for v in V:
    print(f"vehiculo {v}: Tipo={vehicles_df['VehicleType'][v]}, Capacidad={Q[v]}, Rango={R[v]}")

vehiculo 0: Tipo=Gas Car, Capacidad=131.9211396722696, Rango=145.85207096486445
vehiculo 1: Tipo=EV, Capacidad=108.4356199315333, Rango=1304.605971281605
vehiculo 2: Tipo=EV, Capacidad=91.50425520531196, Rango=953.172608610164
vehiculo 3: Tipo=drone, Capacidad=32.896064077536955, Rango=17.302304187458727
vehiculo 4: Tipo=drone, Capacidad=22.65262807032524, Rango=16.627680130757895
vehiculo 5: Tipo=drone, Capacidad=22.682911535937688, Rango=13.602810739289229


# Creacion del modelo

In [434]:
model=ConcreteModel()

#Definicion de conjuntos
model.C= Set(initialize=C) # Centros de distribucion
model.K = Set(initialize=K) # Clientes
model.V = Set(initialize=V) # Vehículos
model.N = model.C | model.K    # Todos los nodos CDs + Clientes el | hace la unicoon de los conjuntos


In [435]:
# Parámetros
model.CAP = Param(model.C, initialize=CAP)
model.DEM = Param(model.K, initialize=DEM)
model.Q = Param(model.V, initialize=Q)
model.R = Param(model.V, initialize=R)

# Distancias entre centros y clientes
model.DIST = Param(model.C, model.K, initialize=DIST, within=NonNegativeReals)

Pf = 123.12  # COP/km (combustible)
Ft = 823     # COP/km (flete)
Cm = 700     # COP/km (mantenimiento)


### Variables de decision

In [436]:
model.x =Var(model.C,model.K,model.V,domain=Binary) #X{i,j,l}
model.y= Var(model.C,model.V,domain=Binary) #y{i,l}
model.I=Var(model.C,domain=NonNegativeReals) # Inventario asignado a los centros de distribucion
model.u= Var(model.K,model.V,domain=NonNegativeReals) # Carga entregada en cada punto de distribucion



### Funcion objetivo

In [437]:
def objetivo(model):
  return sum(
    (Pf+Ft+Cm)* model.DIST[i,j]*model.x[i,j,l]
    for i in model.C for j in model.K for l in model.V
  )

model.OBJ=Objective(rule=objetivo,sense=minimize)


####  Restricciones

Capacidad de los Centros de Distribución

In [438]:
def restriccionCapacidad_CentroDistribucion(model,i):
  return model.I[i]<=model.CAP[i]

model.RestriccionCapacidad_CentroDistribucion=Constraint(model.C, rule=restriccionCapacidad_CentroDistribucion)

Satisfacción de la demanda de cada cliente

In [439]:
def restriccion_demanda(model, j):
    return sum(model.u[j, l] for l in model.V) == model.DEM[j]

model.RestriccionDemanda = Constraint(model.K, rule=restriccion_demanda)

Capacidad del vehículo

In [440]:
def restriccion_capacidad_vehiculo(model, l):
    return sum(model.u[j, l] for j in model.K) <= sum(model.Q[l] * model.y[i, l] for i in model.C)

model.RestriccionCapacidadVehiculo = Constraint(model.V, rule=restriccion_capacidad_vehiculo)


Asignar cada vehículo a un único centro de distribución

In [441]:
def restriccion_asignacion_vehiculos(model, l):
    return sum(model.y[i, l] for i in model.C) == 1

model.RestriccionAsignacionVehiculo = Constraint(model.V, rule=restriccion_asignacion_vehiculos)


Rango util del vehiculo

In [442]:
def restriccion_rango(model, l):
    return sum(model.DIST[i, j] * model.x[i, j, l] for i in model.C for j in model.K) <= model.R[l]

model.RestriccionRango = Constraint(model.V, rule=restriccion_rango)


In [443]:
def link_xy_rule(m, i, j, l):
    return m.x[i, j, l] <= m.y[i, l]
model.link_xy = Constraint(model.C, model.K, model.V, rule=link_xy_rule)

In [444]:
def link_ux_rule(m, j, l):
    return m.u[j, l] <= m.Q[l] * sum(m.x[i, j, l] for i in m.C)
model.link_ux = Constraint(model.K, model.V, rule=link_ux_rule)


In [445]:
def visita_rule(m, j):
    return sum(m.x[i, j, l] for i in m.C for l in m.V) == 1
model.visita = Constraint(model.K, rule=visita_rule)

In [446]:
# 2.1) Cada vehículo parte de exactamente un CD
def origen_rule(m, l):
    return sum(m.y[i, l] for i in m.C) == 1
model.un_origen = Constraint(model.V, rule=origen_rule)

# 2.2) Solo puedo usar x[i,j,l] si y[i,l]=1
def link_xy_rule(m, i, j, l):
    return m.x[i, j, l] <= m.y[i, l]
model.link_xy = Constraint(model.C, model.K, model.V, rule=link_xy_rule)

'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


resolver el modelo

In [447]:
solver = SolverFactory('glpk')
# Resolver
result = solver.solve(model, tee=True)

# Mostrar el estado y el valor de la función objetivo
print("Estado:", result.solver.status)
print("Óptimo:", value(model.OBJ))


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\jorgi\AppData\Local\Temp\tmp_kp0hq3d.glpk.raw --wglp C:\Users\jorgi\AppData\Local\Temp\tmp5t_liwkz.glpk.glp
 --cpxlp C:\Users\jorgi\AppData\Local\Temp\tmpo8v7qyq2.pyomo.lp
Reading problem data from 'C:\Users\jorgi\AppData\Local\Temp\tmpo8v7qyq2.pyomo.lp'...
C:\Users\jorgi\AppData\Local\Temp\tmpo8v7qyq2.pyomo.lp:7342: warning: lower bound of variable 'x2' redefined
C:\Users\jorgi\AppData\Local\Temp\tmpo8v7qyq2.pyomo.lp:7342: warning: upper bound of variable 'x2' redefined
756 rows, 786 columns, 3630 non-zeros
720 integer variables, all of which are binary
8062 lines were read
Writing problem data to 'C:\Users\jorgi\AppData\Local\Temp\tmp5t_liwkz.glpk.glp'...
6645 lines were written
GLPK Integer Optimizer 5.0
756 rows, 786 columns, 3630 non-zeros
720 integer variables, all of which are binary
Preprocessing...
605 constraint coefficient(s) were reduced
676 rows, 707 columns, 3175 non-zeros
653 int

In [448]:
# Ver qué rutas fueron seleccionadas
for i in model.C:
    for j in model.K:
        for l in model.V:
            if model.x[i, j, l].value == 1:        
                print(f"Vehículo {l} va de CD {i} a Cliente {j}")

Vehículo 4 va de CD 4 a Cliente 7
Vehículo 4 va de CD 4 a Cliente 8
Vehículo 5 va de CD 5 a Cliente 1
Vehículo 2 va de CD 5 a Cliente 2
Vehículo 2 va de CD 5 a Cliente 4
Vehículo 1 va de CD 6 a Cliente 6
Vehículo 0 va de CD 7 a Cliente 9
Vehículo 3 va de CD 9 a Cliente 3
Vehículo 3 va de CD 9 a Cliente 5


In [449]:
for i in model.C:
    for l in model.V:
        if model.y[i, l].value == 1:
            print(f"Vehículo {l} parte desde CD {i}")


Vehículo 4 parte desde CD 4
Vehículo 2 parte desde CD 5
Vehículo 5 parte desde CD 5
Vehículo 1 parte desde CD 6
Vehículo 0 parte desde CD 7
Vehículo 3 parte desde CD 9
